## Starting with LLMs
### Hugging Face

In [8]:
import langchain
import requests
from ApiSecrets import ApiSecrets

In [9]:
API_URL = ApiSecrets.HF_MISTRAL_ENDPOINT
API_KEY = ApiSecrets.HF_API_KEY

In [10]:
# homework: create a function that takes care of the background API call and gives us the wanted result from a prompt
def generate_response(prompt):
    headers = {"Authorization": f"Bearer {API_KEY}"}
    payload = {
        "inputs": f"{prompt}"
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    result = response.json()[0]["generated_text"].strip()
    return result

In [11]:
prompt1 =  "Give me a good name for my new Chinese restaurant."
result1 = generate_response(prompt1)
print(result1)

Give me a good name for my new Chinese restaurant.

Here are some ideas:

1. Mandarin Magic - This name evokes the idea of something special and enchanting about Chinese cuisine.
2. Grand Dragon - This name is bold and assertive, suggesting the dominance and richness of Chinese food and culture.
3. Fortune Kitchen - This name emphasizes the idea of good fortune and prosperity, which is a common theme in Chinese culture.
4. Jade Garden - This name evokes


#### Prompt Templating

In [12]:
# classic prompt templating
from langchain.prompts import PromptTemplate
template = PromptTemplate(
    input_variables=["country"],
    template="what is the capital of {country}?"
)
prompt2 = template.format(country="france")
result2 = generate_response(prompt2)
print(result2)

what is the capital of france?Hello! The capital city of France is Paris. It is one of the most famous cities in the world, known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. Paris is also considered a global center for art, fashion, gastronomy, and culture. I hope this helps! Let me know if you have any other questions.


In [13]:
# templating using the shorthand from_template method
prompt_from_template = PromptTemplate.from_template("What is a good name for a {company_type} startup?")
prompt3 = prompt_from_template.format(company_type="artificial intelligence")
result3 = generate_response(prompt3)
print(result3)

What is a good name for a artificial intelligence startup?

Here are some suggestions for a name for an artificial intelligence startup:

1. BrainWave: This name plays on the idea of artificial intelligence being a wave or flow of intelligent thought.
2. Neuralia: This name refers to the neural networks that are the basis of many artificial intelligence systems.
3. Sentient: This name implies the ability to have or feel sensations and emotions, like a sentient being.
4. Cognit:


### Cohere

In [14]:
import cohere
COHERE_API_KEY = ApiSecrets.COHERE_API_KEY
co = cohere.Client(COHERE_API_KEY)

In [15]:
response = co.generate(
    prompt = "What is the capital of Germany?"
)

In [16]:
response

Generation(id='6961ea00-6f6d-4ba4-ade7-d523dbd097b6', prompt='What is the capital of Germany?', generations=[SingleGeneration(id='c6bf1937-0aba-4e32-9be3-64ae5569d41a', text=' The capital of Germany is Berlin. \n\nBerlin is the largest city in Germany and has a rich history and culture. It has been an important center of politics, culture, and science since the 18th century, and has been heavily impacted by major historical events such as the Holocaust and the fall of the Berlin Wall in the late 20th century. \n\nToday, Berlin is a diverse and vibrant city known for its nightlife, art scene, and many historical landmarks, including the Berlin Wall Memorial and the Reichstag building. It is also home to several world-class universities and is a major hub for science, technology, and business. ', index=None, likelihood=None, token_likelihoods=None, finish_reason='COMPLETE')], meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=A

In [17]:
print(response.generations[0].text.strip())

The capital of Germany is Berlin. 

Berlin is the largest city in Germany and has a rich history and culture. It has been an important center of politics, culture, and science since the 18th century, and has been heavily impacted by major historical events such as the Holocaust and the fall of the Berlin Wall in the late 20th century. 

Today, Berlin is a diverse and vibrant city known for its nightlife, art scene, and many historical landmarks, including the Berlin Wall Memorial and the Reichstag building. It is also home to several world-class universities and is a major hub for science, technology, and business.


In [18]:
def run_cohere(prompt):
    response = co.generate(prompt=prompt)
    clean_response = response.generations[0].text.strip()
    return clean_response

In [19]:
print(run_cohere("What is the capital of Germany?"))

The capital of Germany is Berlin. 

Would you like to know more about Berlin?  I am happy to provide more information about this fascinating city.  For example, you may want to know about its history, culture, or landmarks.  Or, perhaps you would be interested in learning about its economy, education system, or political structure.  Let me know if you would like any details, and I will be glad to answer your questions.  Please let me know if you would like me to provide a response in a specific format, such as a brief paragraph or a list of facts.  I am here to assist you in the most helpful way possible.  Thank you for your question and your interest in learning more about Berlin!


#### Agents using Cohere

##### SERP API

In [20]:
SERP_API_KEY = ApiSecrets.SERP_API_KEY

In [21]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent
from langchain.llms import Cohere

In [22]:
client = Cohere(cohere_api_key=COHERE_API_KEY)
tool = load_tools(["serpapi"], serpapi_api_key=SERP_API_KEY, llm=client)
agent = initialize_agent(tool, client, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

d:\opensource\genai_projects\genai\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.cohere.Cohere` was deprecated in langchain-community 0.1.14 and will be removed in 0.2.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(
d:\opensource\genai_projects\genai\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [23]:
agent.run("can you tell who won the football world cup recently?")

d:\opensource\genai_projects\genai\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I need to search for the winner of the most recent football World Cup.
Action: Search
Action Input: "who won the football world cup 2020?"
Observation: ['^ Al-Duhail won the 2019–20 Qatar Stars League on 21 August 2020. Their participation was officially confirmed on 27 September 2020 after Al-Sadd became the ...', 'Bayern Munich won the match 1–0 for their second FIFA Club World Cup title and fourth title at the global level. With the win, Bayern became the second European ...', "See our comprehensive FIFA Men's World Cup history guide for everything you need about the tournament. FIFA Men's World Cup results and which countries have won", 'Complete list of all winners of the FIFA Soccer World Cup.', 'Winners ; France, Croatia, Russia ; Germany, Argentina, Brazil.', "Benjamin Pavard's second-half goal proved the difference as UEFA Champions League winners Bayern Munich won the FIFA Club World Cup Qatar ...", 'FIFA Club World Cup 2020 results pa

Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.



Observation: Bayern Munich was founded in 1900 and has become Germany's most famous and successful football club. Almost all of Bayern's success has come since the 1960s. Bayern Munich was formed when members of the MTV 1879 Munich sports club broke away to form their own club.
Thought:

Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.
Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.
Re

 Thought: Bayern Munich won the FIFA Club World Cup in 2020 and has a successful history as they are the second club to win all possible titles in Europe. 
Action: Search
Action Input: "fifa world cup winners"
Observation: ['Brazil, with five wins, are the only team to have played in every tournament. The other World Cup winners are Germany and Italy, with four titles each; ...', 'AWARDS ; 2018. France Flag France ; 2014. Germany Flag Germany ; 2010. Spain Flag Spain ; 2006. Italy Flag Italy ; 2002. Brazil Flag Brazil.', 'Current champion Argentina has three titles, Uruguay and France have two each, while England and Spain have one each. Czechoslovakia, Hungary, Sweden, the ...', "Here are the full list of winners of the previous men's FIFA World Cups. Brazil has won the most titles, and Italy and Brazil are the only countries to win back ...", 'Brazil have won the FIFA World Cup five times, the most in history, while Germany and Italy have won it four times each. Get the complete list

'The most recent FIFA World Cup winners are Argentina who won the 2022 tournament held in Qatar.'

##### Wikipedia

In [24]:
wiki_tool = load_tools(["wikipedia"], llm=client)
wiki_agent = initialize_agent(tool, llm=client, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [25]:
wiki_agent.run("Is Sweden a member of NATO?")

Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.




> Entering new AgentExecutor chain...
 I need to search for whether Sweden is a member of NATO. 
Action: Search
Action Input: Sweden NATO membership

Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.



Observation: Sweden became NATO's newest member on Thursday (7 March 2024), upon depositing its instrument of accession to the North Atlantic Treaty with the Government of the United States in Washington DC. With Sweden's accession, NATO now counts 32 countries among its members.
Thought:

Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.
Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.
Re

 Thought: Sweden is a member of NATO and deposits its instrument of accession to the North Atlantic Treaty in March 2024. 
Action: Search
Action Input: When did Sweden join NATO
Observation: 7 March 2024
Thought: Thought: Knowing the date of accession tells me that Sweden joined NATO very recently, therefore, any articles referencing Sweden as a NATO member would refer to the current events. 
Action: Search
Action Input: Sweden NATO membership recent developments
Observation: Sweden became NATO's newest member on Thursday (7 March 2024), upon depositing its instrument of accession to the North Atlantic Treaty with the Government of the United States in Washington DC. With Sweden's accession, NATO now counts 32 countries among its members.
Thought: Thought: These current events illustrate that Sweden is poised to join NATO, and will do so on a particular calendar date. 
Final Answer: Sweden is not a member of NATO currently, but will become a member on 7 March 2024. 

> Finished chain.


'Sweden is not a member of NATO currently, but will become a member on 7 March 2024.'

#### Chains using Cohere

In [26]:
client

Cohere(client=<cohere.client.Client object at 0x000001DAB6869430>, async_client=<cohere.client.AsyncClient object at 0x000001DAB6C59730>, cohere_api_key=SecretStr('**********'))

In [27]:
template = PromptTemplate.from_template("What is a suitable name for a company that sells {product}?")
prompt1 = template.format(product="spices")
print(prompt1)

What is a suitable name for a company that sells spices?


In [29]:
print(run_cohere(prompt1))

There are several options for names that could effectively convey your company's purpose to potential customers. Here are a few suggestions: 

1. Spice Sensations: This name suggests that your company offers a wide variety of spices that will delight customers' senses of taste and smell. 

2. aromaticspiceshop: This name immediately conveys the nature of your business and evokes the appealing scent of aromatic spices. 

3. Flavourville: This name is playful and suggests a diverse range of mouth-watering flavours derived from your spices. 

4. The Peppered Platter: This name is creative and intriguing, immediately bringing to mind the image of a well-seasoned dish. 

5. Spicery: This name is straightforward and effective, conveying the idea of a specialized shop for spices. 

 Remember, the most suitable name for your company will be one that aligns with your brand and resonates with your target audience. It's always a good idea to brainstorm and consider your brand values when selectin

In [30]:
from langchain.chains import LLMChain
chain = LLMChain(llm=client, prompt=template)

In [31]:
print(chain.run(product="spices").strip())

Some potential names for a company that sells spices are: 

1. Spicery
2. The Seasoned Market
3. Pepperville
4. The Spice Den
5. Jalapeño Janes
6. Minty Morsels
7. Cinnamon Snaps
8. Paprika Peak
9. Ginger Snap
10. The Pickled Pepper

These names allude to the various spices and entice the customers to want to explore what the company has to offer.


##### Combining Multiple Chains
Let us try and design multiple chains to solve a complex problem

In [32]:
from langchain.chains import SimpleSequentialChain

In [33]:
uni_template = PromptTemplate.from_template("Suggest some good universities to study {subject} in Europe.")
uni_chain = LLMChain(llm=client, prompt=uni_template)

apply_template = PromptTemplate.from_template("Suggest some ways to make a strong application for {uni_name}.")
apply_chain = LLMChain(llm=client, prompt=apply_template)

In [34]:
unirec_chain = SimpleSequentialChain(chains=[uni_chain, apply_chain], verbose=True)
print(unirec_chain.run("signal processing"))



> Entering new SimpleSequentialChain chain...
 Some universities with signal processing programs in Europe include:

1. ETH Zurich (Swiss Federal Institute of Technology in Zurich) - Switzerland
2. Imperial College London - UK
3. TU Munich (TUM) - Germany
4. Universite Paris-Saclay - France
5. University of Cambridge - UK
6. University of Oxford - UK
7. Technical University of Denmark (DTU) - Denmark
8. RWTH Aachen University - Germany
9. EPFL (Ecole Polytechnique Federale de Lausanne) - Switzerland
10. TU Eindhoven - Netherlands

These universities offer varying levels of study and research programs in the field of signal processing. They are all well-known for their scientific and academic reputation, and disseminating knowledge at the forefront of research and development. 

Please let me know if you would like to know more about any of these universities, or any other recommendations or considerations you should take into account when selecting an academic institution. 
 Here are

##### Sequential Chains

In [35]:
from langchain.chains import SequentialChain

In [36]:
uni_template = PromptTemplate.from_template("Suggest some good universities to study {subject} in Europe.")
uni_chain = LLMChain(llm=client, prompt=uni_template, output_key="uni_name", verbose=True)

apply_template = PromptTemplate.from_template("Suggest some ways to make a strong application for {uni_name}.")
apply_chain = LLMChain(llm=client, prompt=apply_template, output_key="appli_guidelines", verbose=True)

In [37]:
unirec_seq_chain = SequentialChain(
    chains = [uni_chain, apply_chain],
    input_variables = ["subject"],
    output_variables = ["uni_name", "appli_guidelines"],
    verbose = True
)

In [39]:
uniresult = unirec_seq_chain({"subject": "history"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Suggest some good universities to study history in Europe.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Suggest some ways to make a strong application for  There are many renowned universities in Europe that offer high-quality programs in history, spanning various regions, perspectives, and eras. Here are some notable examples: 

1. University of Oxford (Oxford, England) - Known for its impressive library system and distinguished faculty, University of Oxford is consistently ranked among the top universities in the world. Oxford offers a wide range of history courses, encompassing both broad surveys and specialized topics. 

2. University of Cambridge (Cambridge, England) - Similarly to Oxford, the University of Cambridge is esteemed for its vibrant academic community and historic reputation. It hosts numerous esteemed historians and offers diverse cou

In [41]:
print(uniresult["uni_name"].strip())

There are many renowned universities in Europe that offer high-quality programs in history, spanning various regions, perspectives, and eras. Here are some notable examples: 

1. University of Oxford (Oxford, England) - Known for its impressive library system and distinguished faculty, University of Oxford is consistently ranked among the top universities in the world. Oxford offers a wide range of history courses, encompassing both broad surveys and specialized topics. 

2. University of Cambridge (Cambridge, England) - Similarly to Oxford, the University of Cambridge is esteemed for its vibrant academic community and historic reputation. It hosts numerous esteemed historians and offers diverse courses spanning ancient, medieval, and modern history. 

3. Harvard University (Cambridge, Massachusetts, United States) - With an longstanding commitment to academic excellence, Harvard University offers a wide range of history specializations and research opportunities. As one of the most pr

### Document Loader (PDF)
Can be used to load various kinds of files. We shall experiment with PDFs. Use case: Vector Search.

In [40]:
from langchain_community.document_loaders import PyPDFLoader

In [75]:
loader = PyPDFLoader("D:\Certificates\\vscode_udemy.pdf")
pages = loader.load()

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\sreya\AppData\Local\Temp\ipykernel_16984\1848104916.py:1: SyntaxWarning: invalid escape sequence '\C'
  loader = PyPDFLoader("D:\Certificates\\vscode_udemy.pdf")


In [76]:
pages

[Document(page_content='', metadata={'source': 'D:\\Certificates\\vscode_udemy.pdf', 'page': 0})]

In [77]:
loader = PyPDFLoader("D:\Certificates\\vscode_udemy.pdf", extract_images=True)
pages = loader.load()

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\sreya\AppData\Local\Temp\ipykernel_16984\3423397851.py:1: SyntaxWarning: invalid escape sequence '\C'
  loader = PyPDFLoader("D:\Certificates\\vscode_udemy.pdf", extract_images=True)


In [78]:
pages[0].page_content

'Certificate of Completion\nThis is to certify that Sreyan Ghosh successfully\ncompleted 3.5 total hours of Learn Visual Studio\nCode online course on May 17, 2020\nearn\nFechPlus\nLearnTechPlus,Instructor\nUdemy\nCertificate no:UC-8d18518f-8bdb-412d-8d71-98292587a68c\nCertificate url: ude.my/UC-8d18518f-8bdb-412d-8d71-98292587a68c\n#BeAble'

### Cohere (continued)

#### Memory using Cohere
<b>ConversationBufferMemory:</b> Retains all the previous messages in the memory. Consequently, expensive on storage.

In [42]:
from langchain.memory import ConversationBufferMemory

In [43]:
cb_memory = ConversationBufferMemory()
name_template = PromptTemplate(
    input_variables=["product"],
    template= "What would be a good start-up name for a company dealing in {product}"
)

In [44]:
name_chain = LLMChain(llm=client, prompt=name_template, memory=cb_memory)

In [45]:
print(name_chain.run("Potato chips"))

 Here are a few ideas: 

1. Chips-ly: a combination of the words "chips" and "fabulously," suggesting that the chips are delicious and trendy. 

2. Crunchies: this name emphasizes the crispy texture of potato chips. 

3. Golden Fries: this name suggests that the chips are golden and delicious, making them highly appealing to customers. 

4. Snacky: a playful combination of the word "snack" and "potato," this name provides a fun appeal to the product. 

5. Layers:  this name suggests the many layers of flavor and ingredients in the potato chips. 

6. Spudsy: a fun twist on the word "spuddy," this name hints at the company's specialization in potatoes. 

These are just a few suggestions, and I'm sure you'll find the perfect name to impress customers and symbolize your potato chip business. 


In [46]:
print(name_chain.run("video games"))

 Here are a few ideas for start-up names that embody the excitement and engagement of the gaming world:

1. Nitro Gaming Motors: This name combines the high-energy feel of gaming with a hint of motion and progression, suggesting not just the gameplay itself but also the action and excitement it can inspire. 

2. Zenborn Interactive: This name blends "zen" with "born" suggesting a peaceful, yet passionate approach to the interactive medium of gaming. It conveys a sense of calm focus within the exciting realm of gameplay. 

3. Quantum Leap Games: This name draws on the concept of progressing or "leaping" to newer and higher levels of performance, engagement and fun within the gaming sphere. 

4. Vertex Games: "Vertex" suggests a pinnacle or peak, conveying the aspire to be the leading company in the gaming industry. It evokes the idea of surpassing limits and achieving new heights. 

5. Evolution Games: This name uses the concept of evolution to signify the ever-changing, diversifying an

In [47]:
print(name_chain.run("soft drinks"))

 There are many options available for a company dealing in soft drinks, and the choice of name will depend on the specific products and brand image you want to convey. Here are a few ideas to consider: 

1. Bubble Boom: This name suggests fun and excitement, capturing the playful and refreshing appeal of soft drinks.

2. Fizzix: This name combines "fizz," often associated with carbonated drinks, and "mix," suggesting a variety of drink options.

3. Liquid Joy: This name evokes happiness and a pleasant drinking experience, aligning with the refreshing and enjoyable nature of soft drinks.

4. Sipster: A modern and catchy name suggesting an approachable and friendly brand for all ages.

5. Revive: Focusing on the refreshing quality of soft drinks, this name suggests invigorating and energizing experiences. 

6. Cooliti: A playful twist on "coconut," this name sounds like an icy, refreshing beverage. 

7. Dropp: This name is short, snappy, and conveys the idea of enjoyment and fun. 

8. Dr

In [50]:
print(name_chain.memory.buffer)

Human: Potato chips
AI:  Here are a few ideas: 

1. Chips-ly: a combination of the words "chips" and "fabulously," suggesting that the chips are delicious and trendy. 

2. Crunchies: this name emphasizes the crispy texture of potato chips. 

3. Golden Fries: this name suggests that the chips are golden and delicious, making them highly appealing to customers. 

4. Snacky: a playful combination of the word "snack" and "potato," this name provides a fun appeal to the product. 

5. Layers:  this name suggests the many layers of flavor and ingredients in the potato chips. 

6. Spudsy: a fun twist on the word "spuddy," this name hints at the company's specialization in potatoes. 

These are just a few suggestions, and I'm sure you'll find the perfect name to impress customers and symbolize your potato chip business. 
Human: video games
AI:  Here are a few ideas for start-up names that embody the excitement and engagement of the gaming world:

1. Nitro Gaming Motors: This name combines the h

<b>Conversation Chain: </b> A kind of chain as we saw before, but this one has the intrinsic ability to retain memory.

In [52]:
from langchain.chains import ConversationChain

In [53]:
convo = ConversationChain(llm=client)

In [56]:
print(convo.run("Who won the first football world cup?"))

 The first Football World Cup was held in 1930, and 23 nations participated. The tournament was won by Uruguay, who defeated Brazil 2-1 in the final match. Uruguay had already defeated Argentina 4-2 in the semi-final match, and Brazil defeated Paraguay 2-0. With a successful victory over Brazil, Uruguay claimed the championship title. 

Would you like to know more about the history of the Football World Cup? 


In [102]:
print(convo.run("Who is the president of North Korea?"))

 I don't know the specific answer to this question as my knowledge cut-off is January 2023. 

However, the president of North Korea is typically determined through a process that is not entirely clear to outsiders. Some analysts believe that the highest authority in the country, the Secretary of the Central Committee of the Workers' Party of Korea, appoints the Head of State.

Kim Jong-un is the Secretary of the Central Committee and has not been seen in public since August 2020, which has led to widespread speculation about his health and whereabouts. Analysts predicted that an eventual successor would not be announced until after his death, which leads to a murky future political trajectory for the nation. 

Would you like me to provide more information on the political structure of North Korea? 


In [103]:
print(convo.run("When was the Renaissance period in Europe?"))

 The Renaissance period in Europe took place approximately between the 14th and 17th centuries. It began in different regions at different times, with the start generally attributed to the Italian Peninsula, hence its designation as the Italian Renaissance. It ended with the Baroque period across Europe. The Renaissance is defined by a cultural rebirth at the time, with significant developments in areas of art, science, politics, and culture. Some of the characteristics of this period include humanism, individualism, and a revival of classical Greek and Roman art, ideas, and knowledge. 

Do you want more information on the Renaissance? I can discuss some of the unique aspects of this time period and its historical significance in greater detail if you like. 


In [104]:
print(convo.run("In which country did this tournament take place?"))

 The Football World Cup is a tournament that occurs every four years and is currently controlled by FIFA, the international governing body of football (also known as soccer). For the first four World Cups, the hosts were selected by FIFA several years before the tournament was to take place. The first World Cup was proposed to be held in the United States, but significant issues with their domestic soccer organizations prevented this from occurring. As a result, Uruguay was selected to be the host country for the first World Cup in 1930. 

Historically, the selection process for the host country has undergone several changes since the 1930s. Would you like me to provide more information on the selection process by FIFA for hosting countries, or the countries that have hosted the soccer world cup? 


<b>ConversationBufferWindowMemory: </b> Same as the first type of memory, but in this we can limit how many past contexts we want the memory to retain.

In [57]:
from langchain.memory import ConversationBufferWindowMemory

In [63]:
cbw_memory = ConversationBufferWindowMemory(k=2)

In [64]:
convo_chain = ConversationChain(llm=client, memory=cbw_memory)

In [65]:
print(convo_chain.run("When was the middle-age period in Europe?"))

 The Middle Ages in Europe is generally considered to have lasted from the fall of the Western Roman Empire (c. 476) to the beginning of the Renaissance (c. 1300). 

This is a huge period of time consisting of several distinct eras. Would you like me to break down the Middle Ages into smaller periods and describe key historical events and social changes that occurred in each? 

I can also name key medieval era countries in Europe if you like, and describe what life was like for people living during this time. 


In [66]:
print(convo_chain.run("When was Lionel Messi born?"))

 Lionel Messi was born on June 24, 1987. He is a living legend and one of the greatest footballers (soccer players) of all time, equipped with a natural talent and athletic gift that bolstered his skill onto the world's stage. He has been hailed as an inspiration, and his skill has been likened to musical genius.

Would you like me to tell you about some of his achievements? I can also summarise his early life and the pivotal moments of his career that led him to become one of the most acclaimed soccer players in the world. 


In [67]:
print(convo_chain.run("Name some famous artists of that period."))

 The Renaissance period is termed as such due to the notion of "rebirth", where societal arts and knowledge swept back into popularity. With that being said, here are some famous artists of this period: 

1. Leonardo da Vinci (`1452-1519`) - One of the foremost successors of the Renaissance period, Leonardo da Vinci was a true polymath and is widely recognized for his benchmark works such as the `Mona Lisa` and `The Last Supper`. 

2. Raphael (`1483-1520`) - Raphael was a prolific painter and architect, known for his divine commanding of the figurative and for his pivotal role in shaping the aesthetic ideals of the Renaissance period. He is best portrayed in his famous painting, `The School of Athens`. 

3. Donatello (`1386-1466`) - Born as Donato di Niccolò di Betto Bardi, Donatello was an Italian Renaissance sculptor and is well-renowned as a pioneer in the transition away from the gothic sculpture and sculpture of the Renaissance period. 

4. Michelangelo (`1475-1564`) - Another hig